In [2]:
import os
import csv
import pandas as pd

Set up
This file....

LAR / HMDA data (state_AZ-CA.csv)
To download the entire file, go to this link. In Step 1 select Arizona and California. Then click download dataset.
Save this file to the /input_data/ folder.
Note: This file is 1.6GB, so it is not in the repo.
Note: There are columns with "mixed types".
The code below will create a mini version to work with while we develop the analysis.

In [2]:
if not os.path.exists("../input_data/state_AZ-CA_mini.csv"):
    hmda = pd.read_csv('../input_data/state_AZ-CA.csv')
    hmda.sample(50000).to_csv('../input_data/state_AZ-CA_mini.csv',index=False)

C:\Users\twsca\AppData\Local\Temp\ipykernel_1156\2645739437.py:2: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44,46,77,78,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  hmda = pd.read_csv('../input_data/state_AZ-CA.csv')


In [3]:
hmda = pd.read_csv('../input_data/state_AZ-CA.csv')

C:\Users\twsca\AppData\Local\Temp\ipykernel_27028\1184079942.py:1: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44,46,77,78,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  hmda = pd.read_csv('../input_data/state_AZ-CA.csv')


In [4]:
# Filter the rows that correspond to denied loans
denied_loans = hmda[hmda['action_taken'] == 3]

# Calculate the number of denied loans
denied_count = len(denied_loans)

# Calculate the total number of loans
total_count = len(hmda)

# Calculate the denial rate
denial_rate = denied_count / total_count

# Print the denial rate
print("Denial Rate:", denial_rate)

Denial Rate: 0.09843446606851623


denied_loans = hmda[hmda['action_taken'] == 3]
grouped = denied_loans.groupby('lei').size().reset_index(name='denied_loans_count')
grouped['total_loans_count'] = hmda.groupby('lei').size()
grouped['denial_rate'] = grouped['denied_loans_count'] / grouped['total_loans_count']

def group_lei(row):
    if row['lei'] == 'QOT5WN9RBKQTFRVKEV31':
        return 'lei_1'
    elif row['lei'] == '3Y4U8VZURTYWI1W2K376':
        return 'lei_2'
    else:
        return 'lei_3'

hmda['grouped_lei'] = hmda.apply(group_lei, axis=1)

print(hmda.head())

group_lei takes one row of the data set and figures out what group it should be in based on the LEI. .appy function takes this function and applies it to every row in thedataset.

grouped = hmda.groupby(['derived_loan_product_type', 'grouped_lei'])
hmda['denial_rate'] = grouped['action_taken'].mean().apply(lambda x: x == 3)




In [5]:
hmda["denied"] = hmda["action_taken"].apply(lambda x: x == 3)

# Calculate the denial rate for each loan product type
loan_types = hmda["derived_loan_product_type"].unique()
denial_rates = {}
for loan_type in loan_types:
    subset = hmda[hmda["derived_loan_product_type"] == loan_type]
    denial_rate = subset["denied"].mean()
    denial_rates[loan_type] = denial_rate

print(denial_rates)
#This will give you a dictionary with the denial rate for each loan product type

{'Conventional:First Lien': 0.08472209710836862, 'FHA:First Lien': 0.10395396719676164, 'FSA/RHS:First Lien': 0.07674913540835329, 'Conventional:Subordinate Lien': 0.2920300392586581, 'VA:First Lien': 0.08865946042645713, 'FHA:Subordinate Lien': 0.026391279403327597, 'VA:Subordinate Lien': 0.015384615384615385, 'FSA/RHS:Subordinate Lien': 0.010526315789473684}


In [6]:
#Groupby code for Bank of the West lei
hmda_lei = hmda.groupby('lei')
hmda_lei.get_group('QOT5WN9RBKQTFRVKEV31')

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,denied
4166027,2021,QOT5WN9RBKQTFRVKEV31,42200,CA,6083.0,6.083002e+09,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,NaN,NaN,5275,46.86,90100,164,1453,1819,41,False
4166028,2021,QOT5WN9RBKQTFRVKEV31,46060,AZ,4019.0,4.019005e+09,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,2143,20.53,68600,220,736,1024,38,False
4166029,2021,QOT5WN9RBKQTFRVKEV31,11244,CA,6059.0,6.059099e+09,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,4534,22.52,106700,101,779,1535,39,False
4166030,2021,QOT5WN9RBKQTFRVKEV31,31084,CA,6037.0,6.037800e+09,NC,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,5129,24.18,80000,219,1623,1909,31,False
4166031,2021,QOT5WN9RBKQTFRVKEV31,38060,AZ,4021.0,4.021000e+09,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Manufactured,Not Hispanic or Latino,...,NaN,NaN,7201,7.60,79000,127,3180,4871,14,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4185454,2021,QOT5WN9RBKQTFRVKEV31,31084,CA,6037.0,6.037240e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,3768,99.97,80000,54,466,1018,70,False
4185455,2021,QOT5WN9RBKQTFRVKEV31,38060,AZ,4013.0,4.013082e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,5500,85.51,79000,50,867,1518,23,False
4185456,2021,QOT5WN9RBKQTFRVKEV31,37100,CA,6111.0,6.111005e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,6845,74.30,98800,67,919,1075,11,False
4185457,2021,QOT5WN9RBKQTFRVKEV31,38060,AZ,4013.0,4.013061e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,5979,25.39,79000,101,1054,1959,11,False


In [7]:
#denial rates for bank of the west

lei = 'QOT5WN9RBKQTFRVKEV31'
hmda_lei_group = hmda_lei.get_group(lei)
loan_product_types = hmda_lei_group['derived_loan_product_type'].unique()
denial_rates = {}
for loan_product_type in loan_product_types:
    loan_product_type_data = hmda_lei_group[hmda_lei_group['derived_loan_product_type'] == loan_product_type]
    num_denials = (loan_product_type_data['action_taken'] == 3).sum()
    total = loan_product_type_data.shape[0]
    denial_rate = num_denials / total
    denial_rates[loan_product_type] = denial_rate
denial_rates_df = pd.DataFrame.from_dict(denial_rates, orient='index', columns=[lei])

In [8]:
print(denial_rates_df)

                               QOT5WN9RBKQTFRVKEV31
Conventional:Subordinate Lien              0.382844
Conventional:First Lien                    0.220618
VA:First Lien                              0.160000
FHA:First Lien                             0.450549


In [9]:
#denial rates for BMO

lei = '3Y4U8VZURTYWI1W2K376'
lei_group = hmda_lei.get_group(lei)
lei_denial_rate = lei_group[lei_group['action_taken'] == 3].groupby('derived_loan_product_type').size() / lei_group.groupby('derived_loan_product_type').size()

In [10]:
print(lei_denial_rate)

derived_loan_product_type
Conventional:First Lien          0.206002
Conventional:Subordinate Lien    0.329078
FHA:First Lien                   0.500000
dtype: float64
